In [37]:
# Imports
import pandas as pd
import numpy as np
import requests
import joblib
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from datetime import datetime, timedelta

print("✅ Imports réussis!")

✅ Imports réussis!


In [38]:
# Récupération des données historiques (5 ans) avec PLUIE
print("📥 Téléchargement des données historiques (5 ans)...")
print("⏳ Inclut température + précipitations...")

end_date = datetime.now().strftime("%Y-%m-%d")
start_date = (datetime.now() - timedelta(days=1826)).strftime("%Y-%m-%d")

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 34.0209,
    "longitude": -6.8416,
    "start_date": start_date,
    "end_date": end_date,
    "hourly": "temperature_2m,precipitation,relative_humidity_2m,cloud_cover",
    "timezone": "GMT"
}

response = requests.get(url, params=params)
data = response.json()

print(f"✅ Données récupérées du {start_date} au {end_date}")
print(f"📊 Variables: température, précipitations, humidité, nuages")

📥 Téléchargement des données historiques (5 ans)...
⏳ Inclut température + précipitations...
✅ Données récupérées du 2020-12-04 au 2025-12-04
📊 Variables: température, précipitations, humidité, nuages
✅ Données récupérées du 2020-12-04 au 2025-12-04
📊 Variables: température, précipitations, humidité, nuages


In [39]:
# Préparation des données (Température + Pluie)
print("🔧 Préparation des données...")

df = pd.DataFrame({
    'date': pd.to_datetime(data['hourly']['time']),
    'temperature': data['hourly']['temperature_2m'],
    'precipitation': data['hourly']['precipitation'],
    'humidity': data['hourly']['relative_humidity_2m'],
    'cloud_cover': data['hourly']['cloud_cover']
})

# Création des features temporelles
df['jour_annee'] = df['date'].dt.dayofyear
df['mois'] = df['date'].dt.month
df['heure'] = df['date'].dt.hour

# Encodage cyclique (Sin/Cos)
df['day_cos'] = np.cos(2 * np.pi * df['jour_annee'] / 365.25)
df['day_sin'] = np.sin(2 * np.pi * df['jour_annee'] / 365.25)
df['hour_cos'] = np.cos(2 * np.pi * df['heure'] / 24)
df['hour_sin'] = np.sin(2 * np.pi * df['heure'] / 24)
df['month_cos'] = np.cos(2 * np.pi * df['mois'] / 12)
df['month_sin'] = np.sin(2 * np.pi * df['mois'] / 12)

# Variable cible pour la pluie (1 = pluie, 0 = pas de pluie)
df['pluie'] = (df['precipitation'] > 0).astype(int)

df = df.dropna()

print(f"✅ {len(df)} observations préparées")
print(f"🌧️ Jours avec pluie: {df['pluie'].sum()} ({df['pluie'].mean()*100:.1f}%)")
df.head()

🔧 Préparation des données...
✅ 43848 observations préparées
🌧️ Jours avec pluie: 3932 (9.0%)


,date,temperature,precipitation,humidity,cloud_cover,jour_annee,mois,heure,day_cos,day_sin,hour_cos,hour_sin,month_cos,month_sin,pluie
0,2020-12-04 00:00:00,16.2,0.0,91,56,339,12,0,0.899766,-0.436373,1.000000,0.000000,1.0,-2.449294e-16,0
1,2020-12-04 01:00:00,15.8,0.2,92,57,339,12,1,0.899766,-0.436373,0.965926,0.258819,1.0,-2.449294e-16,1
2,2020-12-04 02:00:00,15.7,0.2,91,48,339,12,2,0.899766,-0.436373,0.866025,0.500000,1.0,-2.449294e-16,1
3,2020-12-04 03:00:00,15.6,0.1,91,30,339,12,3,0.899766,-0.436373,0.707107,0.707107,1.0,-2.449294e-16,1
4,2020-12-04 04:00:00,15.4,0.1,92,34,339,12,4,0.899766,-0.436373,0.500000,0.866025,1.0,-2.449294e-16,1


In [40]:
# Entraînement des modèles (Température + Pluie)
print("🧠 Entraînement des modèles...")

features = ['day_cos', 'day_sin', 'hour_cos', 'hour_sin', 'month_cos', 'month_sin', 'heure', 'mois', 'jour_annee']
X = df[features]

# 1. Modèle de température
print("   → Modèle température...")
y_temp = df['temperature']
model_temp = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
model_temp.fit(X, y_temp)

# 2. Modèle de pluie (classification)
print("   → Modèle pluie...")
y_pluie = df['pluie']
model_pluie = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
model_pluie.fit(X, y_pluie)

print("✅ Les 2 modèles sont entraînés!")

🧠 Entraînement des modèles...
   → Modèle température...
   → Modèle pluie...
   → Modèle pluie...
✅ Les 2 modèles sont entraînés!
✅ Les 2 modèles sont entraînés!


In [41]:
# Sauvegarde des modèles
import os

save_path = r'C:\Users\User\OneDrive\Bureau\projet1111\cerveau_meteo_long_terme.pkl'

# On sauvegarde les 2 modèles ensemble
model_data = {
    'model_temp': model_temp,
    'model_pluie': model_pluie,
    'features': features
}
joblib.dump(model_data, save_path)

print(f"💾 Modèles sauvegardés: {save_path}")
print(f"   → Modèle température ✅")
print(f"   → Modèle pluie ✅")

if os.path.exists(save_path):
    print("✅ Fichier créé avec succès!")

💾 Modèles sauvegardés: C:\Users\User\OneDrive\Bureau\projet1111\cerveau_meteo_long_terme.pkl
   → Modèle température ✅
   → Modèle pluie ✅
✅ Fichier créé avec succès!


In [42]:
# Test des modèles
print("🧪 Test des modèles...")

test_data = pd.DataFrame({
    'jour_annee': [datetime.now().timetuple().tm_yday],
    'mois': [datetime.now().month],
    'heure': [14]
})
test_data['day_cos'] = np.cos(2 * np.pi * test_data['jour_annee'] / 365.25)
test_data['day_sin'] = np.sin(2 * np.pi * test_data['jour_annee'] / 365.25)
test_data['hour_cos'] = np.cos(2 * np.pi * test_data['heure'] / 24)
test_data['hour_sin'] = np.sin(2 * np.pi * test_data['heure'] / 24)
test_data['month_cos'] = np.cos(2 * np.pi * test_data['mois'] / 12)
test_data['month_sin'] = np.sin(2 * np.pi * test_data['mois'] / 12)

# Prédictions
pred_temp = model_temp.predict(test_data[features])[0]
prob_pluie = model_pluie.predict_proba(test_data[features])[0][1] * 100  # Probabilité de pluie en %

print(f"\n📅 Prédiction pour aujourd'hui à 14h à Rabat:")
print(f"🌡️ Température: {pred_temp:.1f}°C")
print(f"🌧️ Probabilité de pluie: {prob_pluie:.0f}%")
print("\n✅ TERMINÉ! Vous pouvez maintenant relancer app_meteo.py")

🧪 Test des modèles...

📅 Prédiction pour aujourd'hui à 14h à Rabat:
🌡️ Température: 18.5°C
🌧️ Probabilité de pluie: 5%

✅ TERMINÉ! Vous pouvez maintenant relancer app_meteo.py
